In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('/home/niklas/projects/pytorch-cnn-visualizations/')
from src.cnn_layer_visualization import CNNLayerVisualization
import torch, torchvision
import os
from torchvision.models import resnet18, resnet50

In [2]:
model_type = 'resnet18'
image_representation = 'rgb'
use_jpeg = False
datetime = {'resnet50': {'rgb': {True: '2023-02-23-17:21:11', False: '2023-02-23-17:21:11'}, 'coc': {True: '2023-02-23-17:21:18', False: '2023-02-23-17:21:29'}},
            'resnet18': {'rgb': {True: '2023-03-01-18:07:32', False: '2023-03-01-18:04:17'}, 'coc': {True: '2023-03-01-18:08:01', False: '2023-03-01-18:08:05'}}}

model_path = os.path.join('/home/niklas/projects/oads_results_snellius/', model_type, image_representation, "jpeg" if use_jpeg else "", datetime[model_type][image_representation][use_jpeg])
model_path = os.path.join(model_path, [x for x in os.listdir(model_path) if x.startswith('best_model')][0])

initial_output_channels = 19


output_channels = 565 # 200 # len(dataset.decision_mapping.categories)

torch.cuda.empty_cache()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [8]:
model = resnet18()

model.conv1 = torch.nn.Conv2d(in_channels=3, out_channels=model.conv1.out_channels, kernel_size=(7,7), stride=(2,2), padding=(3,3), bias=False)
model.fc = torch.nn.Linear(in_features=512, out_features=initial_output_channels, bias=True)
model.load_state_dict(torch.load(model_path))
for child in model.children():
    for param in child.parameters():
        param.requires_grad = False

# model.fc = torch.nn.Linear(in_features=512, out_features=output_channels, bias=True)
# model = model.to(device)
# modules = list(model.children())[:-1]      # delete the last fc layer.
# model = torch.nn.Sequential(*modules)
# model.features = torch.nn.Sequential(*modules)

In [9]:
for x, layer in enumerate(model):
    print(layer)

TypeError: 'ResNet' object is not iterable

In [10]:
layer_vis = CNNLayerVisualization(model=model, selected_layer=1, selected_filter=5)
layer_vis.visualise_layer_with_hooks()

TypeError: 'ResNet' object is not subscriptable

In [11]:
from src.gradcam import GradCam
from src.misc_functions import get_example_params, save_class_activation_images

In [17]:
list(model._modules.items())[0]

('conv1',
 Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False))

In [19]:
target_example = 0  # Snake
(original_image, prep_img, target_class, file_name_to_export, pretrained_model) =\
    get_example_params(target_example)
# Grad cam
grad_cam = GradCam(model, target_layer='conv1')
# Generate cam mask
cam = grad_cam.generate_cam(prep_img, target_class)
# Save mask
save_class_activation_images(original_image, cam, file_name_to_export)
print('Grad cam completed')

RuntimeError: mat1 and mat2 shapes cannot be multiplied (512x1 and 512x19)